### score-0.6340802988

In [262]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

#train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'TIMESTAMP'])
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

train_x['y0'] = 0
train_x['y1'] = 0
train_x['y2'] = 0
test_x['y0'] = 0
test_x['y1'] = 0
test_x['y2'] = 0


In [263]:
from sklearn.preprocessing import LabelEncoder

qual_col = ['LINE','PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[i] = le.transform(test_x[i])

print('done')

done


In [264]:
## PRODUCT_CODE

# train
train_x_1 = train_x[train_x['PRODUCT_CODE'] == 0]#.drop('Y_Class', axis=1)
train_x_2 = train_x[train_x['PRODUCT_CODE'] == 1]#.drop('Y_Class', axis=1)
train_x_3 = train_x[train_x['PRODUCT_CODE'] == 2]#.drop('Y_Class', axis=1)

train_y_1 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 0]
train_y_2 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 1]
train_y_3 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 2]

print('train_x_1 shape :', train_x_1.shape,
      '\ntrain_x_2 shape :', train_x_2.shape,
      '\ntrain_x_3 shape :', train_x_3.shape)

# test
test_x_1 = test_x[test_x['PRODUCT_CODE'] == 0]
test_x_2 = test_x[test_x['PRODUCT_CODE'] == 1]
test_x_3 = test_x[test_x['PRODUCT_CODE'] == 2]

print('done')

train_x_1 shape : (249, 2881) 
train_x_2 shape : (6, 2881) 
train_x_3 shape : (343, 2881)
done


In [265]:
# LINE

## TRAIN

# line 1
train_x_1_1 = train_x_1[train_x_1['LINE'] == 0]

# line 2
train_x_1_2 = train_x_1[train_x_1['LINE'] == 1]

# line 3
train_x_1_3 = train_x_1[train_x_1['LINE'] == 2]

# line 4
train_x_1_4 = train_x_1[train_x_1['LINE'] == 3]

# line 5
train_x_2_5 = train_x_2[train_x_2['LINE'] == 4]
train_x_3_5 = train_x_3[train_x_3['LINE'] == 4]

# line 6
train_x_2_6 = train_x_2[train_x_2['LINE'] == 5]
train_x_3_6 = train_x_3[train_x_3['LINE'] == 5]

train_set = [train_x_1_1, train_x_1_2, train_x_1_3, train_x_1_4, train_x_2_5, train_x_3_5, train_x_2_6, train_x_3_6]

for set in train_set:
    print(set.shape)

(59, 2881)
(70, 2881)
(78, 2881)
(42, 2881)
(3, 2881)
(172, 2881)
(3, 2881)
(171, 2881)


In [266]:
## TEST

# line 1
test_x_1_1 = test_x_1[test_x_1['LINE'] == 0]

# line 2
test_x_1_2 = test_x_1[test_x_1['LINE'] == 1]

# line 3
test_x_1_3 = test_x_1[test_x_1['LINE'] == 2]

# line 4
test_x_1_4 = test_x_1[test_x_1['LINE'] == 3]

# line 5
test_x_2_5 = test_x_2[test_x_2['LINE'] == 4]
test_x_3_5 = test_x_3[test_x_3['LINE'] == 4]

# line 6
test_x_2_6 = test_x_2[test_x_2['LINE'] == 5]
test_x_3_6 = test_x_3[test_x_3['LINE'] == 5]

test_set = [test_x_1_1, test_x_1_2, test_x_1_3, test_x_1_4, test_x_2_5, test_x_3_5, test_x_2_6, test_x_3_6]

for set in test_set:
    print(set.shape)

(14, 2880)
(14, 2880)
(13, 2880)
(26, 2880)
(3, 2880)
(108, 2880)
(1, 2880)
(131, 2880)


In [270]:
for idx, set in enumerate(train_set):
    sum = set['Y_Class'].value_counts().sum()
    y_value = set['Y_Class'].value_counts()
    per = (y_value/sum).sort_index()

    data = pd.DataFrame(data=per, columns=['per'], index=[0,1,2])
    data.per = 0

    for i in per.index:
        data.loc[i]=per[i]
    
    set['y0'] = data.per[0]
    set['y1'] = data.per[1]
    set['y2'] = data.per[2]
    
    test_set[idx]['y0'] = data.per[0]
    test_set[idx]['y1'] = data.per[1]
    test_set[idx]['y2'] = data.per[2]


test_set = [test_x_1_1, test_x_1_2, test_x_1_3, test_x_1_4, test_x_2_5, test_x_3_5, test_x_2_6, test_x_3_6]
train_set = [train_x_1_1, train_x_1_2, train_x_1_3, train_x_1_4, train_x_2_5, train_x_3_5, train_x_2_6, train_x_3_6]

In [271]:
train_x = pd.concat(train_set, axis=0).sort_index()

train_x.drop('Y_Class', axis=1, inplace=True)

train_x.shape

(598, 2880)

In [272]:
from sklearn.model_selection import train_test_split

x_t, x_v, y_t, y_v = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [273]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(x_t, y_t)
pred = xgb.predict(x_v)

from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(pred, y_v))
print(confusion_matrix(pred, y_v))

0.725
[[ 7  2  1]
 [16 71 11]
 [ 0  3  9]]


In [261]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(train_x, train_y)
pred = xgb.predict(test_x)


submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = pred
submit_csv.to_csv('XGBoost_null_per.csv', index=False)